# Modeling

In [9]:
from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import random
import ast


In [10]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings
from sklearn.metrics import classification_report
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV


In [11]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

from tqdm import tqdm

tqdm.pandas()

In [16]:
def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    return x_train, x_test, y_train, y_test

In [52]:
# v2 csv
df_cdc_clean = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')

# filter to moms
df_cdc_clean = df_cdc_clean[df_cdc_clean['has_been_pregnant'] == 1]

df_cdc_clean

/Users/jonhand/Desktop/w210_Capstone/capstone210-ppd-carmeli_chari_hand_lee/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (664,665) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SEQN,SMQ681,SMQ690A,SMQ710,SMQ720,SMQ725,SMQ690B,SMQ740,SMQ690C,SMQ770,...,live_birth_count,age_at_first_birth,age_at_last_birth,months_since_birth,horomones_not_bc,smoked_100_cigs,currently_smoke,height_in,weight_lbs,attempt_weight_loss_1yr
8,109284,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,17.0,24.0,NaN,2.0,2.0,NaN,60.0,178.0,1.0
11,109290,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2.0,2.0,NaN,63.0,155.0,1.0
12,109291,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,21.0,39.0,NaN,2.0,2.0,NaN,64.0,148.0,2.0
15,109295,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,18.0,24.0,NaN,2.0,2.0,NaN,63.0,137.0,2.0
18,109300,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,30.0,33.0,NaN,2.0,2.0,NaN,59.0,130.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29140,83700,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,27.0,NaN,2.0,2.0,NaN,64.0,153.0,2.0
29141,83701,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,26.0,NaN,2.0,2.0,NaN,65.0,160.0,2.0
29144,83711,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,27.0,30.0,NaN,2.0,2.0,NaN,61.0,185.0,NaN
29146,83717,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,19.0,40.0,NaN,2.0,2.0,NaN,NaN,93.0,2.0


In [53]:
cols_to_keep = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless', 
    'trouble_falling_or_staying_asleep', 
    'feeling_tired_or_having_little_energy', 
    'poor_appetitie_or_overeating', 
    'feeling_bad_about_yourself', 
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast', 
    'thoughts_you_would_be_better_off_dead', 
    'difficult_doing_daytoday_tasks',
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'count_lost_10plus_pounds', 
    'arthritis',
    'horomones_not_bc', 
    'is_usa_born', 
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule', 
    'age_in_years',
    'regular_periods',
    'count_minutes_moderate_sedentary_activity', 
    'emergency_food_received',
    'high_bp',
    'dr_recommend_exercise', 
    'metal_objects', 
    'drank_alc', 
    'cholesterol_prescription',
    'smoked_100_cigs',
    'vigorous_recreation', 
    'dr_recommend_lose_weight',
    'cancer', 
    'chest_discomfort',
    'has_health_insurance', 
    'have_health_insurance', 
    'weight_lbs',
    'readytoeat_meals', 
    'regular_healthcare_place', 
    'try_pregnancy_1yr', 
    'currently_increase_exercise', 
    'coronary_heart_disease', 
    'stroke',
    'heart_attack',
    'see_dr_fertility']
len(cols_to_keep)

46

In [54]:
df_cdc_clean = df_cdc_clean[['MDD'] + cols_to_keep]
df_cdc_clean

,MDD,little_interest_in_doing_things,feeling_down_depressed_hopeless,trouble_falling_or_staying_asleep,feeling_tired_or_having_little_energy,poor_appetitie_or_overeating,feeling_bad_about_yourself,trouble_concentrating,moving_or_speaking_to_slowly_or_fast,thoughts_you_would_be_better_off_dead,...,have_health_insurance,weight_lbs,readytoeat_meals,regular_healthcare_place,try_pregnancy_1yr,currently_increase_exercise,coronary_heart_disease,stroke,heart_attack,see_dr_fertility
8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,178.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0
11,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,155.0,3.0,1.0,NaN,1.0,2.0,2.0,2.0,NaN
12,0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,0.0,...,1.0,148.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
15,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,137.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0
18,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,130.0,12.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29140,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,153.0,12.0,1.0,2.0,NaN,2.0,2.0,2.0,2.0
29141,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,160.0,0.0,1.0,2.0,NaN,2.0,2.0,2.0,2.0
29144,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,185.0,0.0,1.0,2.0,NaN,2.0,2.0,2.0,2.0
29146,0,3.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,...,2.0,93.0,0.0,2.0,NaN,NaN,2.0,2.0,2.0,NaN


In [69]:
# testing code
print(df_cdc_clean.iloc[1,:].to_dict())

{'MDD': 0.0, 'little_interest_in_doing_things': 1.0, 'feeling_down_depressed_hopeless': 1.0, 'trouble_falling_or_staying_asleep': 0.0, 'feeling_tired_or_having_little_energy': 0.0, 'poor_appetitie_or_overeating': 0.0, 'feeling_bad_about_yourself': 0.0, 'trouble_concentrating': 0.0, 'moving_or_speaking_to_slowly_or_fast': 0.0, 'thoughts_you_would_be_better_off_dead': 0.0, 'difficult_doing_daytoday_tasks': 1.0, 'times_with_12plus_alc': nan, 'seen_mental_health_professional': 2.0, 'count_days_seen_doctor_12mo': 1.0, 'count_lost_10plus_pounds': nan, 'arthritis': 1.0, 'horomones_not_bc': 2.0, 'is_usa_born': 1.0, 'times_with_8plus_alc': nan, 'time_since_last_healthcare': nan, 'duration_last_healthcare_visit': nan, 'work_schedule': 2.0, 'age_in_years': 68.0, 'regular_periods': 2.0, 'count_minutes_moderate_sedentary_activity': 180.0, 'emergency_food_received': 2.0, 'high_bp': 1.0, 'dr_recommend_exercise': 1.0, 'metal_objects': 2.0, 'drank_alc': 1.0, 'cholesterol_prescription': 1.0, 'smoked_100

In [3]:
for k,v in  {
    "little_interest_in_doing_things": 1.0,
    "feeling_down_depressed_hopeless": 1.0,
    "trouble_falling_or_staying_asleep": 0.0,
    "feeling_tired_or_having_little_energy": 0.0,
    "poor_appetitie_or_overeating": 0.0,
    "feeling_bad_about_yourself": 0.0,
    "trouble_concentrating": 0.0,
    "moving_or_speaking_to_slowly_or_fast": 0.0,
    "thoughts_you_would_be_better_off_dead": 0.0,
    "difficult_doing_daytoday_tasks": 1.0,
    "times_with_12plus_alc": None,
    "seen_mental_health_professional": 2.0,
    "count_days_seen_doctor_12mo": 1.0,
    "count_lost_10plus_pounds": None,
    "arthritis": 1.0,
    "horomones_not_bc": 2.0,
    "is_usa_born": 1.0,
    "times_with_8plus_alc": None,
    "time_since_last_healthcare": None,
    "duration_last_healthcare_visit": None,
    "work_schedule": 2.0,
    "age_in_years": 68.0,
    "regular_periods": 2.0,
    "count_minutes_moderate_sedentary_activity": 180.0,
    "emergency_food_received": 2.0,
    "high_bp": 1.0,
    "dr_recommend_exercise": 1.0,
    "metal_objects": 2.0,
    "drank_alc": 1.0,
    "cholesterol_prescription": 1.0,
    "smoked_100_cigs": 2.0,
    "vigorous_recreation": 2.0,
    "dr_recommend_lose_weight": 2.0,
    "cancer": 2.0,
    "chest_discomfort": 2.0,
    "has_health_insurance": 1.0,
    "have_health_insurance": 1.0,
    "weight_lbs": 155.0,
    "readytoeat_meals": 3.0,
    "regular_healthcare_place": 1.0,
    "try_pregnancy_1yr": None,
    "currently_increase_exercise": 1.0,
    "coronary_heart_disease": 2.0,
    "stroke": 2.0,
    "heart_attack": 2.0,
    "see_dr_fertility": None,
}.items():
    print(f'{k}: float | None = None,')

little_interest_in_doing_things: float | None = None,
feeling_down_depressed_hopeless: float | None = None,
trouble_falling_or_staying_asleep: float | None = None,
feeling_tired_or_having_little_energy: float | None = None,
poor_appetitie_or_overeating: float | None = None,
feeling_bad_about_yourself: float | None = None,
trouble_concentrating: float | None = None,
moving_or_speaking_to_slowly_or_fast: float | None = None,
thoughts_you_would_be_better_off_dead: float | None = None,
difficult_doing_daytoday_tasks: float | None = None,
times_with_12plus_alc: float | None = None,
seen_mental_health_professional: float | None = None,
count_days_seen_doctor_12mo: float | None = None,
count_lost_10plus_pounds: float | None = None,
arthritis: float | None = None,
horomones_not_bc: float | None = None,
is_usa_born: float | None = None,
times_with_8plus_alc: float | None = None,
time_since_last_healthcare: float | None = None,
duration_last_healthcare_visit: float | None = None,
work_schedule: 

In [55]:
# # SEQN and MDD are the first two columns in the df, so exclude from X
# X = df_cdc_clean.iloc[:,2:].values
# y = df_cdc_clean['MDD'].values


# # X_train, X_test, y_train, y_test = train_test_split(
# #     X, y, test_size=0.2, random_state=42
# # )

In [56]:
# algo_name = 'Logistic Regression'

# X = df_cdc_clean[keep_feats]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# my_imputer = SimpleImputer()
# X_train = my_imputer.fit_transform(X_train)
# X_test = my_imputer.fit_transform(X_test)

# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_resample(X_train, y_train)


In [57]:
y_train

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [1, 1],
       [0, 0],
       [0, 0]])

,0,1,accuracy,macro avg,weighted avg
precision,0.954623,0.278215,0.78825,0.616419,0.885192
recall,0.802158,0.666667,0.78825,0.734412,0.788250
f1-score,0.871775,0.392593,0.78825,0.632184,0.822588
support,1390.000000,159.000000,0.78825,1549.000000,1549.000000


In [66]:

x_train, x_test, y_train, y_test = get_model_data(
    original_df = df_cdc_clean,
    columns = cols_to_keep
)

processing_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), RobustScaler(), GaussianNB())

model_filename = "model_pipeline.pkl"
model_path = join(getcwd(), model_filename)
if not exists(model_path):
    processing_pipeline.fit(x_train, y_train)
    pred_labels  = processing_pipeline.predict(x_test)
    pred_labels = [x.round() for x in pred_labels]
    df_gnb = pd.DataFrame(classification_report(
                y_test, 
                pred_labels, 
                # target_names=target_names, 
                output_dict=True
    ))
    joblib.dump(processing_pipeline, model_path)
    print('successfully trained model')
else:
    print("Model has already been trained, no need to rerun")

df_gnb

Model has already been trained, no need to rerun


,0,1,accuracy,macro avg,weighted avg
precision,0.954623,0.278215,0.78825,0.616419,0.885192
recall,0.802158,0.666667,0.78825,0.734412,0.788250
f1-score,0.871775,0.392593,0.78825,0.632184,0.822588
support,1390.000000,159.000000,0.78825,1549.000000,1549.000000


In [ ]:
try:
    !rm trainer.py
except:
    pass

rm: trainer.py: No such file or directory


In [ ]:
try:
    !jupyter nbconvert --no-prompt --to script trainer.ipynb
except:
    pass

[NbConvertApp] Converting notebook trainer.ipynb to script
[NbConvertApp] Writing 26351 bytes to trainer.py
